In [1]:
!pip install PyPDF2
!pip install PyPDF2 pytesseract Pillow pdf2image
!pip install pytesseract
!pip install PyMuPDF pytesseract pillow


In [4]:
import os
import pandas as pd
from PyPDF2 import PdfReader
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'



# Configurações
pasta = "------CAMINHO DA PASTA--------"
nome_arquivo = "PDF_TESTE.pdf"
caminho_arquivo = os.path.join(pasta, nome_arquivo)

# Lista para armazenar o texto extraído
texto_extraido = []

def extrair_texto_pdf(pdf_path):
    # Tenta extrair texto usando PyPDF2
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            if text:  # Se texto for encontrado, adiciona à lista
                texto_extraido.append(text)
            else:  # Se não houver texto, tenta usar OCR
                texto_extraido.append(extrair_texto_imagens(pdf_path, page_num))

def extrair_texto_imagens(pdf_path, page_num):
    # Abre o PDF e extrai imagens usando PyMuPDF
    pdf_document = fitz.open(pdf_path)
    page = pdf_document[page_num]
    imagens_texto = ""

    # Extrai as imagens da página
    imagens = page.get_images(full=True)
    
    for img_index, img in enumerate(imagens):
        img_index = img[0]
        base_image = pdf_document.extract_image(img_index)
        image_bytes = base_image["image"]

        # Converte bytes em imagem PIL
        image = Image.open(io.BytesIO(image_bytes))
        
        # Realiza o OCR na imagem
        texto_imagem = pytesseract.image_to_string(image)
        imagens_texto += texto_imagem + "\n"

    pdf_document.close()
    return imagens_texto.strip()  # Remove espaços em branco

# Extraindo texto do PDF
extrair_texto_pdf(caminho_arquivo)

# Cria um DataFrame a partir da lista de textos
df = pd.DataFrame({'Texto': texto_extraido})

# Define o caminho do diretório para salvar o arquivo Excel
desktop_path = os.path.join(pasta, "Desktop")
os.makedirs(desktop_path, exist_ok=True)

# Define o caminho do arquivo Excel
caminho_excel = os.path.join(desktop_path, "INFORMAÇÕES.xlsx")

# Salva o DataFrame em um arquivo Excel
df.to_excel(caminho_excel, sheet_name='INFORMAÇÕES', index=False)
print(f"Informações salvas em: {caminho_excel}")

Informações salvas em: C:\Users\jhene\Downloads\PDF_EXTRATOR\Desktop\INFORMAÇÕES.xlsx
